In [3]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State#
import base64

# Configure OS routines
import os#

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



# importing my CRUD Python module file named project_one_crud and class name AnimalShelter
from project_one_crud import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# Update with your username and password and CRUD Python module name. 

username = "aacuser" # username for CRUD module project_one_crud
password = "Kenny#2024" # password for CRUD module project_one_crud

# Connect to database via CRUD Module
shelter = AnimalShelter(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'Grazioso Salvare Logo.png' # Grazioso Salvare Logo image path
encoded_image = base64.b64encode(open(image_filename, 'rb').read()) 


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(
    src='data:image/png;base64,{}'.format(encoded_image.decode()),
    style={"width" : "200px", "display" : "block", "margin" : "0 auto"}
),
    html.Center(html.B(html.H1('Kenneth Wilkerson\'s CS 340 Project 2 Dashboard'))), # personalizing dashboard name
    html.Hr(),
    html.Div([
    html.Label("Select a Filter:"), # Prompt to choose a filter
    dcc.RadioItems( # Interactive filtering options
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'}, # Water Rescue radio button
                {'label': 'Mountain or Wilderness Rescue', 'value': "Mountain or Wilderness Rescue"}, # Mountain or Wilderness Rescue radio button
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'}, # Disaster or Individual Tracking radio button
                {'label': 'Reset', 'value': 'Reset'} # Reset radio button
            ],
            value='Reset',
            labelStyle={'display': 'Block'} # Block style display
        )
    ]),
    dash_table.DataTable( # Interactive data table
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        
        # setting up features for interactive data table
        row_selectable = "single", # ensure single row selection
        selected_rows = [0], # default select first row
        page_size = 10, # limit table to 10 rows per page
        style_table = {'height' : '400px', 'overflowY' : 'auto'}, # make table scrollable 
        style_cell = {'textAlign' : 'center', 'padding' : '5px'}, # centering text
        style_data = {'whiteSpace' : 'normal', 'height' : 'auto'}, # enable text wrapping
        sort_action = "native", #enable column sorting
        sort_mode = "multi", #allow sorting multiple columns

    ),
    html.Br(),
     html.Hr(),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
     html.Div(
            id='map-id',
            className='col s12 m6',
            )
])])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback to update the data table based on chosen filter
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    
    valid_filters = ['Reset', 'Water Rescue', 'Mountain or Wilderness Rescue', 'Disaster or Individual Tracking'] 
    if filter_type not in valid_filters: # Validate filter input
        raise ValueError(f"Invalid Filter Type: {filter_type}")
    
    # Define filter criteria based on selected option
    if filter_type == 'Reset':
        criteria = {} # empty criteria to return all documents
        query = shelter.read(criteria)
    elif filter_type == 'Water Rescue':
        # Water Rescue criteria defined by Grazioso Salvare
        criteria = {"animal_type": "Dog", "breed":{"$in":["Labrador Retriever Mix", "Chesa Bay Retr","Newfoundland"]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte":26}, "age_upon_outcome_in_weeks": {"$lte":156}}
        query = shelter.read(criteria) # Returns matching water rescue dogs
    elif filter_type == 'Mountain or Wilderness Rescue':
        # Mountain or Wilderness Rescue criteria defined by Grazioso Salvare
        criteria = {"animal_type": "Dog", "breed":{"$in":["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":26}, "age_upon_outcome_in_weeks": {"$lte":156}}
        query = shelter.read(criteria) # Returns matching mountain or wilderness rescue dogs
    elif filter_type == 'Disaster or Individual Tracking':
        # Disaster or Individual Tracking criteria defined by Grazioso Salvare
        criteria = {"animal_type": "Dog", "breed": {"$in":["Doberman Pinsch", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":20}, "age_upon_outcome_in_weeks": {"$lte":300}}
        query = shelter.read(criteria) # Returns matching disaster or individual tracking dogs
    
    try:
        df = pd.DataFrame.from_records(query) # Convert query result into a DataFrame
        if '_id' in df.columns: #Drop the '_id' column if it exists
            df.drop(columns=['_id'], inplace=True)
        return df.to_dict('records') # Return updated data
    except Exception as e:
        print(f"Error filtering data: {e}") # Print error message
        return [] # Return empty list for error
    
    



# Callback to update the graph based on the selected data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None: # Check if data is available
        return "No data available"
    
    dff = pd.DataFrame.from_dict(viewData) # Convert data into DataFrame
    if 'breed' not in dff.columns: # Check if breed column exists
        return None
    
    # Calculate outcome counts for pie chart
    outcome_counts = dff['outcome_type'].value_counts()
    
    # Return pie chart
    return[
        dcc.Graph(figure=px.pie(outcome_counts,
                               names=outcome_counts.index,
                               values=outcome_counts.values,
                               title='Distribution of Animal Outcomes'),)]

## This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')] # changed to reflect selected_rows
)
def update_styles(selected_rows):
    if not selected_rows:
        selected_rows = [] # if no row is selected initialize with an empty list
    return [{'if': {'row_index': selected_rows[0]}, # highlight selected row with background color
             'background_color': '#D2F3FF',
             'color' : 'black'}]
        



# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    if not viewData:
        return "No data available" # if viewData is not available return status message
    
    
    #convert viewData into a DataFrame
    dff = pd.DataFrame.from_dict(viewData)
    
    if not index or dff.empty: #validate data and selection
        return "No selection made" 
    
    #Get the selected row (single selection enabled)
    row = index[0]
    
    try:
        #Extract latitude and longitude 
        latitude = dff.iloc[row]["location_lat"] # using column name to locate latitude data
        longitude = dff.iloc[row]["location_long"] # using column name to locate longitude data
        
        #Extract breed and animal name 
        breed = dff.iloc[row]["breed"] # using column name to locate breed data
        animal_name = dff.iloc[row]["name"] # using column name to locate name data
    
    except(KeyError, IndexError):
        return "Invalid data format" # error message if data format is wrong
        
    
    # Return map 
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                  dl.TileLayer(id="base-layer-id"),
                  #Marker with tool tip and popup
                  dl.Marker(position=[latitude, longitude], children=[
                               dl.Tooltip(breed),
                               dl.Popup([
                                   html.H1("Animal Name"),
                                   html.P(animal_name)
                               ])
                           ])
              ])
    ]
    
# Run Dash app
app.run_server(debug=True)

Dash app running on http://127.0.0.1:31222/
